In [1]:
print('Hello World')

Hello World


In [2]:
import pandas as pd

In [5]:

# load the dataset from hugging face
image_caption_df = pd.read_parquet("hf://datasets/SKyu/my-image-captioning-dataset/data/train-00000-of-00001-3e9e72b53b09ab09.parquet")

In [6]:
image_caption_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3100 entries, 0 to 3099
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   prompt       3100 non-null   object
 1   image.bytes  3100 non-null   object
 2   image.path   3100 non-null   object
dtypes: object(3)
memory usage: 72.8+ KB


In [8]:
image_caption_df.head()

,prompt,image.bytes,image.path
0,"a large building next to a body of water, conc...",b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x02\x00...,2010 Yeosu EXPO Korean Pavilion_P_002.jpg
1,"a model of a city next to a body of water, det...",b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x02\x00...,2010 Yeosu EXPO Korean Pavilion_P_003.jpg
2,a group of people standing inside of a buildin...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x02\x00...,2010 Yeosu EXPO Korean Pavilion_P_004.jpg
3,a large building sitting on top of a lush gree...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x02\x00...,2018 PyeongChang Olympic Plaza_201803080906011...
4,a stadium filled with lots of people watching ...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x02\x00...,2018 PyeongChang Olympic Plaza_201803081908528...


In [11]:
from datasets import load_dataset, Dataset
import pandas as pd

dataset = load_dataset("M-CLIP/ImageCaptions-7M-Translations", streaming = True)

sampled_data = []
for i, example in enumerate(dataset):
    if i >= 1000:
        break
    sampled_data.append(example)

# Convert to Hugging Face Dataset object
small_dataset = Dataset.from_list(sampled_data)

# Convert to Pandas DataFrame
df = small_dataset.to_pandas()

# Keep only 'caption' and 'url' columns (drop others)
df = df[['caption', 'url']]

# Save to CSV in current directory
df.to_csv("captions_and_urls.csv", index=False)

print("CSV file saved as captions_and_urls.csv")


AttributeError: 'str' object has no attribute 'get'

In [75]:
# load the the caption_url csv
df = pd.read_csv("caption_url.csv")
print(df)

                                              caption  \
0         an empty chair sitting in front of a window   
1   a house under construction with a building bei...   
2            a little boy that is sitting in a basket   
3         a man in an apron working on crates of fish   
4   a view from a bridge over a river in central park   
..                                                ...   
95  santa claus, standing at the base of a christm...   
96      an old white house with a tree in front of it   
97  the stained glass window in the cathedral at t...   
98  three men in their firemen's uniforms standing...   
99                 a lake with water in the mountains   

                                                  url  
0   http://static.flickr.com/2723/4385058960_b0f29...  
1   http://static.flickr.com/3074/2965635243_d45c3...  
2   http://static.flickr.com/188/431442759_e0c519a...  
3   http://static.flickr.com/3276/2989279316_9e8e3...  
4   http://static.flickr.com/2691/4

In [68]:
df.head()

,caption,url
0,an empty chair sitting in front of a window,http://static.flickr.com/2723/4385058960_b0f29...
1,a house under construction with a building bei...,http://static.flickr.com/3074/2965635243_d45c3...
2,a little boy that is sitting in a basket,http://static.flickr.com/188/431442759_e0c519a...
3,a man in an apron working on crates of fish,http://static.flickr.com/3276/2989279316_9e8e3...
4,a view from a bridge over a river in central park,http://static.flickr.com/2691/4304191133_cbbf8...


In [19]:
# load the model for translation
import torch
from transformers import pipeline

pipeline = pipeline(
    task="translation",
    model="Helsinki-NLP/opus-mt-en-ur")

Device set to use cpu


In [76]:
# create a new dataset for the updated caption and cosine thingy
new_df = pd.DataFrame(columns=['eng_caption', 'urdu_caption', 'url'])

In [21]:
new_df.describe()

,eng_caption,urdu_caption,url
count,0,0,0
unique,0,0,0
top,NaN,NaN,NaN
freq,NaN,NaN,NaN


In [60]:
# load the embedder
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

model = SentenceTransformer('sentence-transformers/LaBSE')


In [61]:
def translate_caption(en_caption, translator):
    result = translator(en_caption, max_length=512)
    return result[0]['translation_text']

In [64]:
def compute_similarity(en_caption, lang_caption, model):
    embeddings = model.encode([en_caption, lang_caption])
    similarity = cosine_similarity(embeddings[0].reshape(1, -1) , embeddings[1].reshape(1, -1))[0][0]
    return similarity

In [77]:
# iterate over the original data frame
# get translation against each row, calculate its similarity
# if similarity score is above 0.75, append it to the new dataframe else ignore
for index, row in df.iterrows():
    translated_caption = translate_caption(row['caption'], pipeline)
    similarity = compute_similarity(row['caption'], translated_caption, model)
    # print(translated_caption)
    # print(similarity)
    if similarity >= 0.75:
        new_row = pd.DataFrame({"url": [row['url']], "eng_caption": [row['caption']], "urdu_caption": [translated_caption]})
        new_df = pd.concat([new_df, new_row], ignore_index=True)


In [78]:
new_df.head()

,eng_caption,urdu_caption,url
0,an empty chair sitting in front of a window,کھڑکی کے سامنے تختوں پر ایک چھوٹا سا تخت بیٹھت...,http://static.flickr.com/2723/4385058960_b0f29...
1,a house under construction with a building bei...,تعمیر کے تحت تعمیر کا گھر تعمیر کِیا گیا تھا,http://static.flickr.com/3074/2965635243_d45c3...
2,a little boy that is sitting in a basket,ایک چھوٹا لڑکا ایک ٹوکری میں بیٹھا ہے,http://static.flickr.com/188/431442759_e0c519a...
3,a man in an apron working on crates of fish,مچھلیوں میں ایک آدمی مچھلی کے جال میں کام کرتا ہے,http://static.flickr.com/3276/2989279316_9e8e3...
4,a view from a bridge over a river in central park,وسطی پارک کے ایک سرے سے لیکر وسطی پارک کا منظر,http://static.flickr.com/2691/4304191133_cbbf8...


In [ ]:
# save the dataframe as a csv
df.to_csv('translated_captions.csv')